In [1]:
# import helper libraries 
import mido # for music handeling 
import numpy as np # for matrix maths 
from sklearn.preprocessing import StandardScaler # for normalization 
from mido import MidiFile, MidiTrack, Message # mido classes to handle music 

# keras library 
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout # keras layers 
from tensorflow.keras.models import Sequential # sequential model 
from tensorflow.keras.preprocessing import sequence # to preprocess sequential data 
from tensorflow.keras.optimizers import RMSprop # for optimizing our model

In [2]:
# process mid file 
mid = MidiFile('./allegroconspirito.mid') # audio training file

notes = [] # notes for midi file 

# intial params 
time = float(0)
prev = float(0)

for msg in mid:
    time += msg.time # to get time of note 
    # only get music data not meta data 
    if not msg.is_meta:
        # only intrested in piano channel 
        if msg.channel == 0:
            if msg.type == 'note_on':
                note = msg.bytes() # get notes data in form of bytes 
                note = note[1:3] # only get note and velocity of our model 
                note.append(time - prev)
                prev = time # set prev to current time 
                notes.append(note)

In [3]:
# scale notes data in form 
t = []
for note in notes:
    note[0] = (note[0]-24)/ 88 # scale 1d of notes
    note[1] = (note[1]) / 127 # scale 2d of notes 
    t.append(note[2])
max_t = max(t) # max of note[2]
for note in notes:
    note[2] = note[2] / max_t # scale note[2]

In [4]:
# create data, labels for predictions 
X = []
Y = []
n_prev = 30 # num of notes to predict 
for i in range(len(notes) - n_prev):
    x = notes[i:i+n_prev] # value of notes as features
    y = notes[i+n_prev] # labels of notes 
    X.append(x) # make a dict of x 
    Y.append(y) # make list of y

# save a seed to do predictions
seed = notes[0:n_prev]

# print values of x and y 
print("Value of X {}".format(X[:5]))
print("Value of y {}".format(Y[:5]))

Value of X [[[0.5113636363636364, 0.47244094488188976, 0.0], [0.5681818181818182, 0.5748031496062992, 0.0], [0.4772727272727273, 0.47244094488188976, 0.0], [0.23863636363636365, 0.44881889763779526, 0.0], [0.1590909090909091, 0.44881889763779526, 0.0], [0.29545454545454547, 0.5433070866141733, 0.0], [0.29545454545454547, 0.0, 0.39423964035494696], [0.5681818181818182, 0.0, 0.0], [0.4772727272727273, 0.0, 0.0], [0.5113636363636364, 0.0, 0.0], [0.1590909090909091, 0.0, 0.0], [0.23863636363636365, 0.0, 0.0], [0.625, 0.5196850393700787, 0.39423964035494696], [0.4772727272727273, 0.3543307086614173, 0.0], [0.4318181818181818, 0.29133858267716534, 0.0], [0.6136363636363636, 0.4330708661417323, 0.09455966033522857], [0.625, 0.0, 0.009455966033522868], [0.5909090909090909, 0.4251968503937008, 0.08510369430170571], [0.6136363636363636, 0.0, 0.009455966033522868], [0.6136363636363636, 0.4251968503937008, 0.0851036943017056], [0.5909090909090909, 0.0, 0.009455966033522868], [0.4772727272727273, 0

In [10]:
# convert data into np arrays 
X_arr = np.array(X, dtype=np.float32)
Y_arr = np.array(Y, dtype=np.float32)

In [11]:
# build model 
print("Building Model.....")
model = Sequential() # initialize a sequential model 
model.add(LSTM(128,return_sequences=True, input_shape=(n_prev, 3))) # 1st layer of LSTM 
model.add(Dropout(0.2)) # 1st layer of Dropout 
model.add(LSTM(64,return_sequences=False, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
model.add(Dropout(0.2)) # 2nd layer of Dropout
model.add(Dense(3)) # a dense layer of 3 tensors 
model.add(Activation('linear'))

# optimizer RMSprop
optimizer = RMSprop(lr=0.01)

# compile model
print("Compiling Model......")
model.compile(loss='mse', optimizer='rmsprop')

Building Model.....
Compiling Model......


In [12]:
model.fit(X_arr,Y_arr, batch_size=300, epochs=200, verbose=1)

Epoch 1/200
5910/5910 [==============================] - 14s 2ms/step - loss: 0.0265
Epoch 2/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0209
Epoch 3/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0205
Epoch 4/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0198
Epoch 5/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0197
Epoch 6/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0191
Epoch 7/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0186
Epoch 8/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0186
Epoch 9/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0182
Epoch 10/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0180
Epoch 11/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0178
Epoch 12/200
5910/5910 [==============================] - 9s 1ms/step - l

5910/5910 [==============================] - 9s 1ms/step - loss: 0.0118
Epoch 98/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0117
Epoch 99/200
5910/5910 [==============================] - 9s 2ms/step - loss: 0.0115
Epoch 100/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0115
Epoch 101/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0113
Epoch 102/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0113
Epoch 103/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0111
Epoch 104/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0112
Epoch 105/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0110
Epoch 106/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0110
Epoch 107/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0108
Epoch 108/200
5910/5910 [==============================] - 9s 1ms/step

5910/5910 [==============================] - 9s 1ms/step - loss: 0.0047
Epoch 193/200
5910/5910 [==============================] - 9s 2ms/step - loss: 0.0046
Epoch 194/200
5910/5910 [==============================] - 11s 2ms/step - loss: 0.0047
Epoch 195/200
5910/5910 [==============================] - 9s 2ms/step - loss: 0.0046
Epoch 196/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0045
Epoch 197/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0046
Epoch 198/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0044
Epoch 199/200
5910/5910 [==============================] - 9s 1ms/step - loss: 0.0044
Epoch 200/200
5910/5910 [==============================] - 8s 1ms/step - loss: 0.0043


In [13]:
# save model into h5 format
model.save('music_model.h5')

In [15]:
# save model into json format
model_json = model.to_json() # make json of model 
# filewriter for storing model as json
with open('music_model.json','w') as file:
    file.write(model_json)

In [16]:
model.evaluate(X_arr[:10], Y_arr[:10])

10/10 [==============================] - 0s 31ms/step


0.0015361630357801914

In [20]:
# make predictions 
predictions = []
x = np.array(seed)
x = np.expand_dims(x, axis=0)

for i in range(3000):
    preds = model.predict(x)
    print(preds)
    x = np.squeeze(x)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axis=0)
    preds = np.squeeze(preds)
    predictions.append(preds)

[[ 0.62575966 -0.00276747  0.0407085 ]]
[[0.6701907  0.39945954 0.07175657]]
[[ 0.6329363  -0.02967735  0.04382139]]
[[0.44033885 0.03044328 0.05353675]]
[[ 0.47789067 -0.00619189  0.01405701]]
[[0.67388177 0.5590726  0.05297973]]
[[ 0.45512462  0.31413394 -0.01613875]]
[[0.50007945 0.28828996 0.00243111]]
[[ 0.69331074 -0.03923395  0.08795995]]
[[0.6963995  0.07802551 0.22903642]]
[[0.5891874  0.05726221 0.02201715]]
[[0.47540152 0.07864453 0.08216297]]
[[0.6638547  0.32248527 0.38198417]]
[[0.61507255 0.01505753 0.0466059 ]]
[[0.60179186 0.23721045 0.09554   ]]
[[ 0.6483587  -0.02217928  0.07538873]]
[[0.7173411  0.3761624  0.11481723]]
[[ 0.6942542  -0.01914565  0.07449979]]
[[0.7030047  0.39488852 0.11887799]]
[[0.59232855 0.10025897 0.06838162]]
[[0.5550571  0.22878778 0.12240934]]
[[0.41194528 0.20498456 0.07724691]]
[[0.37184727 0.04267868 0.14154693]]
[[0.4592875  0.1161651  0.04899118]]
[[0.50823903 0.06893506 0.06384099]]
[[0.5383289  0.05558481 0.04325908]]
[[0.53765327 0.21

[[0.55343246 0.07618032 0.03009117]]
[[0.5603776 0.1377224 0.0303579]]
[[0.5494251  0.028194   0.04770228]]
[[0.56971586 0.18890682 0.00978873]]
[[0.5532464  0.01654622 0.0577388 ]]
[[0.57085955 0.19064535 0.00208369]]
[[0.5517393  0.03404084 0.05835908]]
[[0.56397766 0.15340495 0.00928704]]
[[0.550498   0.07003754 0.05174939]]
[[0.554743   0.09015689 0.02672165]]
[[0.557699   0.12781613 0.03471676]]
[[0.54965466 0.03314526 0.04516957]]
[[0.5684031  0.18436277 0.01199741]]
[[0.55382365 0.01383314 0.05673213]]
[[0.5709211  0.19279383 0.00089946]]
[[0.55224437 0.03058478 0.05820537]]
[[0.5653057  0.15900098 0.00716286]]
[[0.5505714  0.06612732 0.05265518]]
[[0.5556831  0.09800394 0.02477521]]
[[0.5555338  0.12161084 0.03704776]]
[[0.5482591  0.04003981 0.04350832]]
[[0.5657148  0.17729633 0.01518444]]
[[0.5523899  0.01517393 0.05549171]]
[[0.5696177  0.19024038 0.00188907]]
[[0.5529006  0.02711414 0.05812204]]
[[0.5657743  0.1609222  0.00543832]]
[[0.5512955  0.06135959 0.05340385]]
[[0.

[[0.5658727  0.16651969 0.0048865 ]]
[[0.5519705  0.05405171 0.05476068]]
[[0.55877507 0.11083743 0.01941255]]
[[0.5545999  0.10633375 0.04124836]]
[[0.5498472  0.04931098 0.0396151 ]]
[[0.56506145 0.17131142 0.01968735]]
[[0.549312   0.01821367 0.05392577]]
[[0.5696867  0.19439822 0.00448034]]
[[0.55281377 0.02343296 0.05843932]]
[[0.56676733 0.17132366 0.00397461]]
[[0.5520923  0.04971864 0.05563932]]
[[0.55994725 0.11847713 0.01725803]]
[[0.5537877  0.09923179 0.04335   ]]
[[0.5507243  0.05561776 0.03744064]]
[[0.5640535  0.16458587 0.02229757]]
[[0.54907775 0.01979993 0.05271672]]
[[0.5697255  0.19422764 0.00549555]]
[[0.5531146  0.02120705 0.05840085]]
[[0.5677109  0.1757429  0.00315088]]
[[0.55218303 0.04559772 0.05630664]]
[[0.5610693  0.12591478 0.01518374]]
[[0.5531767  0.09251343 0.04532098]]
[[0.55168366 0.06223517 0.03524314]]
[[0.562871   0.15771759 0.02494642]]
[[0.5487584  0.02192689 0.05143699]]
[[0.56951785 0.19363965 0.0067169 ]]
[[0.5532475  0.01918951 0.05831623]]
[

[[0.5686965  0.19228786 0.00907828]]
[[0.5526749  0.01650618 0.05809641]]
[[0.5691232  0.18653008 0.00174793]]
[[0.5519996  0.03599196 0.05772789]]
[[0.5630779  0.14396536 0.01016082]]
[[0.55210125 0.07630464 0.04981774]]
[[0.5541299  0.07960979 0.02931837]]
[[0.55938375 0.13954212 0.03143298]]
[[0.5478152  0.02897836 0.04785631]]
[[0.56802225 0.19103786 0.01033124]]
[[0.5518948  0.01594274 0.05770888]]
[[0.56904536 0.18867217 0.00180382]]
[[0.55198896 0.0336363  0.05797642]]
[[0.56348777 0.14853378 0.00892749]]
[[0.55184734 0.07181118 0.05089041]]
[[0.5547894  0.08503896 0.02748985]]
[[0.55821085 0.13365246 0.03330503]]
[[0.5476263  0.03185602 0.04658198]]
[[0.5674279  0.18872781 0.01180967]]
[[0.55118084 0.0159158  0.0571708 ]]
[[0.5689205  0.19000244 0.00210052]]
[[0.5520898  0.03165604 0.05811366]]
[[0.5639745  0.15256624 0.00790345]]
[[0.5517792  0.06759633 0.05180717]]
[[0.55561674 0.09050967 0.02569576]]
[[0.55728084 0.12761122 0.03510249]]
[[0.54784393 0.03509092 0.04520161]]
[

[[0.56507283 0.1608168  0.00604636]]
[[0.55181223 0.0596946  0.05355665]]
[[0.55740577 0.10198264 0.02209743]]
[[0.55568683 0.11536051 0.03864925]]
[[0.5488617  0.04266748 0.04212645]]
[[0.5659652  0.17833875 0.0167854 ]]
[[0.54983073 0.01689284 0.05518521]]
[[0.56948274 0.1936543  0.00344042]]
[[0.552534   0.02606462 0.05840176]]
[[0.5657654  0.16538408 0.00509657]]
[[0.55190253 0.05534254 0.05450605]]
[[0.5584792  0.10896128 0.02002501]]
[[0.5547922  0.10839133 0.04069167]]
[[0.5495816  0.04783902 0.04019652]]
[[0.56523234 0.17285271 0.01904567]]
[[0.54944193 0.01785822 0.05420503]]
[[0.56961584 0.194131   0.00421112]]
[[0.5527882  0.02397066 0.05842366]]
[[0.5665239  0.16998036 0.00415501]]
[[0.5520973  0.05089974 0.0554268 ]]
[[0.5596508  0.11634764 0.01784147]]
[[0.55397934 0.1013472  0.04276236]]
[[0.5504222  0.05376413 0.03809599]]
[[0.56431437 0.16674465 0.02153617]]
[[0.54907596 0.01927401 0.05309478]]
[[0.5696504  0.19434151 0.005177  ]]
[[0.5530107  0.0218223  0.05841123]]
[

...

[[0.56914234 0.19311501 0.00804828]]
[[0.5530008  0.01755959 0.05821973]]
[[0.56890965 0.18371898 0.00199765]]
[[0.5521514  0.03836381 0.05739199]]
[[0.5627072  0.13931379 0.01141905]]
[[0.5523764  0.08042856 0.04869863]]
[[0.5535552  0.07491063 0.03092012]]
[[0.56040967 0.14447998 0.0297528 ]]
[[0.5480604  0.02690294 0.04886806]]
[[0.5685782  0.19204214 0.00931483]]
[[0.5525527  0.01636735 0.05802669]]
[[0.5691316  0.18698446 0.00174307]]
[[0.55199945 0.03550742 0.05778489]]
[[0.5631543  0.1449294  0.00990311]]
[[0.55205107 0.07538003 0.05004983]]
[[0.5542635  0.08072039 0.02894888]]
[[0.5591392  0.13835217 0.03181951]]
[[0.5477758  0.02953143 0.04760294]]
[[0.56791174 0.19059907 0.01061914]]
[[0.55176747 0.01590618 0.0576061 ]]
[[0.5690255  0.18896161 0.00184678]]
[[0.5520115  0.03322117 0.05800708]]
[[0.5635925  0.14937326 0.00871228]]
[[0.5518196  0.07096396 0.05107981]]
[[0.5549483  0.08613326 0.02713419]]
[[0.5580158  0.1324605  0.03366781]]
[[0.5476554  0.03247628 0.04631476]]
[

[[0.5651982  0.16168657 0.00586053]]
[[0.55182517 0.05885237 0.05374046]]
[[0.55760574 0.10328866 0.02170202]]
[[0.5555101  0.11402325 0.03903956]]
[[0.5489866  0.04360794 0.0417676 ]]
[[0.5658358  0.17733994 0.01720503]]
[[0.5497395  0.01706387 0.05500481]]
[[0.5695079  0.19375761 0.00358391]]
[[0.55258286 0.02567028 0.05840807]]
[[0.56590146 0.16628358 0.00490832]]
[[0.55193657 0.05446135 0.05469266]]
[[0.55870426 0.11039305 0.01959758]]
[[0.5546238  0.10699134 0.04110046]]
[[0.5497402  0.04895028 0.03979141]]
[[0.5650686  0.17170353 0.01951926]]
[[0.5493658  0.01809816 0.05399637]]
[[0.5696331  0.19421223 0.00438496]]
[[0.5528336  0.0235513  0.05842382]]
[[0.56669414 0.17082658 0.00399842]]
[[0.55210865 0.05011402 0.05555677]]
[[0.55986416 0.11769353 0.01745028]]
[[0.5538639  0.10007449 0.04313187]]
[[0.5505923  0.05492067 0.03769927]]
[[0.56412035 0.16551721 0.02201436]]
[[0.54900634 0.0196213  0.05286989]]
[[0.56963766 0.19427623 0.0053905 ]]
[[0.55305123 0.02144223 0.05840024]]
[

[[0.54842675 0.02366506 0.05053164]]
[[0.5692549  0.1933323  0.00767667]]
[[0.5530739  0.01801492 0.05825419]]
[[0.5687823  0.18255545 0.00212339]]
[[0.55219597 0.03932767 0.05724756]]
[[0.5625265  0.1374193  0.01192867]]
[[0.55250067 0.08209392 0.04823915]]
[[0.55332    0.07304901 0.03154812]]
[[0.56080437 0.14645989 0.02907441]]
[[0.54814756 0.02610725 0.04926223]]
[[0.56877613 0.19241652 0.00892666]]
[[0.55275005 0.01661894 0.05812665]]
[[0.5691173  0.18619734 0.00176289]]
[[0.5520134  0.03630918 0.0576876 ]]
[[0.5630354  0.14335637 0.01032919]]
[[0.5521319  0.0768801  0.04966988]]
[[0.55404854 0.07895019 0.02954566]]
[[0.55953157 0.14024343 0.03119974]]
[[0.54785156 0.02866557 0.04800312]]
[[0.5680902  0.19125354 0.01016255]]
[[0.5519907  0.01596686 0.05776612]]
[[0.56906414 0.1884784  0.00177609]]
[[0.5519825  0.03388929 0.05795468]]
[[0.56342983 0.14802387 0.00905783]]
[[0.55186707 0.07233634 0.0507727 ]]
[[0.55469847 0.08438956 0.02770732]]
[[0.5583305  0.13437267 0.03308528]]
[

[[0.5665704  0.1832085  0.01467588]]
[[0.5503543  0.01622315 0.05607969]]
[[0.5692638  0.1927184  0.00277827]]
[[0.55230844 0.02820417 0.05833092]]
[[0.56501096 0.1603883  0.00613715]]
[[0.5518099  0.06010688 0.05346685]]
[[0.5573101  0.10134532 0.02229047]]
[[0.5557737  0.11601494 0.038459  ]]
[[0.5487991  0.04221828 0.04230124]]
[[0.56602305 0.17881872 0.01658554]]
[[0.54986936 0.01681899 0.05527187]]
[[0.5694666  0.19359358 0.00337601]]
[[0.5525113  0.02625583 0.05839844]]
[[0.56569815 0.16493635 0.00518915]]
[[0.55189157 0.05576847 0.05441504]]
[[0.5583728  0.10825823 0.02023172]]
[[0.55487907 0.10907981 0.04049124]]
[[0.54950607 0.04729673 0.04039448]]
[[0.56531334 0.17342903 0.01881433]]
[[0.5494741  0.01774658 0.054308  ]]
[[0.56960595 0.194097   0.00412941]]
[[0.55276394 0.02418101 0.05842331]]
[[0.5664376  0.16955782 0.00423459]]
[[0.5520921  0.05129268 0.05535828]]
[[0.55954504 0.11567304 0.01803617]]
[[0.55404305 0.1019763  0.04257672]]
[[0.5503409  0.0531961  0.03829042]]
[

[[0.56231153 0.15469678 0.02611714]]
[[0.5485053  0.02306309 0.05085945]]
[[0.56934226 0.19348235 0.00735171]]
[[0.553128   0.01843245 0.05827959]]
[[0.56865394 0.18157107 0.00224585]]
[[0.5522218  0.04017317 0.05711982]]
[[0.56235284 0.13578793 0.01237416]]
[[0.5526006  0.08353193 0.04783844]]
[[0.5531054  0.07147066 0.03208202]]
[[0.5611275  0.14812867 0.02849031]]
[[0.54822457 0.02545616 0.0495904 ]]
[[0.5689256  0.19268787 0.00859989]]
[[0.5528567  0.01695229 0.05816165]]
[[0.56905425 0.18530682 0.00184124]]
[[0.55207044 0.0370432  0.05758326]]
[[0.5629279  0.14191127 0.01071855]]
[[0.55221575 0.07814848 0.0493236 ]]
[[0.55387396 0.07749496 0.03004224]]
[[0.55985457 0.1417593  0.03068182]]
[[0.54793215 0.02802415 0.04831572]]
[[0.5682744  0.19154331 0.00985315]]


In [21]:
# to filter predictions 
for pred in predictions:
    pred[0] = int(88*pred[0] + 24)
    pred[1] = int(127*pred[1])
    pred[2] *= max_t
    
    # reject values that would be out of the range 
    if pred[0] < 24:
        pred[0] = 24
    elif pred[0] > 102:
        pred[0] = 102
    if pred[1] < 0:
        pred[1] = 0
    elif pred[1] > 127:
        pred[1] = 127
    if pred[2] < 0:
        pred[2] = 2

In [23]:
# save track into mid file 
mid = MidiFile()
track = MidiTrack()

mid.tracks.append(track)

for note in predictions:
    note = np.insert(note, 0, 147)
    bytes = note.astype(int)
    print(note)
    msg = Message.from_bytes(bytes[0:3])
    time = int(note[3] / 0.001025) # to rescale to midi's delta ticks. arbitary for now 
    msg.time = time 
    track.append(msg)
    
mid.save('new_song.mid') # save song into a track

[1.4700000e+02 7.9000000e+01 0.0000000e+00 4.3682583e-02]
[1.4700000e+02 8.2000000e+01 5.0000000e+01 7.6998964e-02]
[1.4700000e+02 7.9000000e+01 0.0000000e+00 4.7022898e-02]
[1.4700000e+02 6.2000000e+01 3.0000000e+00 5.7448033e-02]
[1.4700000e+02 6.6000000e+01 0.0000000e+00 1.5083992e-02]
[1.470000e+02 8.300000e+01 7.100000e+01 5.685032e-02]
[147.  64.  39.   2.]
[1.4700000e+02 6.8000000e+01 3.6000000e+01 2.6087216e-03]
[1.470000e+02 8.500000e+01 0.000000e+00 9.438612e-02]
[147.          85.           9.           0.24576937]
[1.4700000e+02 7.5000000e+01 7.0000000e+00 2.3625676e-02]
[1.470000e+02 6.500000e+01 9.000000e+00 8.816563e-02]
[147.         82.         40.          0.4098912]
[1.470000e+02 7.800000e+01 1.000000e+00 5.001083e-02]
[1.4700000e+02 7.6000000e+01 3.0000000e+01 1.0251997e-01]
[1.470000e+02 8.100000e+01 0.000000e+00 8.089648e-02]
[1.4700000e+02 8.7000000e+01 4.7000000e+01 1.2320555e-01]
[1.470000e+02 8.500000e+01 0.000000e+00 7.994259e-02]
[1.4700000e+02 8.5000000e+01

[1.470000e+02 7.400000e+01 2.400000e+01 7.301071e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2600166e-02]
[1.4700000e+02 7.3000000e+01 2.2000000e+01 2.8094577e-03]
[1.4700000e+02 7.2000000e+01 5.0000000e+00 6.0950577e-02]
[1.4700000e+02 7.3000000e+01 1.6000000e+01 1.4283054e-02]
[1.470000e+02 7.200000e+01 1.000000e+01 5.031157e-02]
[1.4700000e+02 7.2000000e+01 8.0000000e+00 3.5630986e-02]
[1.4700000e+02 7.3000000e+01 1.9000000e+01 2.9136553e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.4010775e-02]
[1.470000e+02 7.400000e+01 2.400000e+01 8.589818e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2522195e-02]
[1.470000e+02 7.400000e+01 2.300000e+01 2.347512e-03]
[1.470000e+02 7.200000e+01 4.000000e+00 6.158451e-02]
[1.4700000e+02 7.3000000e+01 1.7000000e+01 1.2400438e-02]
[1.4700000e+02 7.2000000e+01 1.0000000e+01 5.2178465e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 3.3232793e-02]
[1.470000e+02 7.300000e+01 1.800000e+01 3.177986e-02]
[1.4700000e+02 7.2000000e+01 3.0000000

[1.4700000e+02 7.2000000e+01 1.1000000e+01 5.0129272e-02]
[1.4700000e+02 7.2000000e+01 8.0000000e+00 3.5977248e-02]
[1.4700000e+02 7.3000000e+01 1.9000000e+01 2.8852236e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.4147962e-02]
[1.470000e+02 7.400000e+01 2.400000e+01 8.310297e-03]
[1.470000e+02 7.200000e+01 2.000000e+00 6.250367e-02]
[1.4700000e+02 7.4000000e+01 2.3000000e+01 2.2528563e-03]
[1.4700000e+02 7.2000000e+01 4.0000000e+00 6.1459642e-02]
[1.4700000e+02 7.3000000e+01 1.7000000e+01 1.2698168e-02]
[1.4700000e+02 7.2000000e+01 1.0000000e+01 5.1856555e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 3.3727497e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 3.1335067e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.2782524e-02]
[1.470000e+02 7.400000e+01 2.400000e+01 9.655199e-03]
[1.470000e+02 7.200000e+01 2.000000e+00 6.236449e-02]
[1.4700000e+02 7.4000000e+01 2.3000000e+01 1.8731942e-03]
[1.470000e+02 7.200000e+01 4.000000e+00 6.192576e-02]
[1.4700000e+02 7.3000000e+01 1.800

[1.4700000e+02 7.4000000e+01 2.3000000e+01 2.3594804e-03]
[1.4700000e+02 7.2000000e+01 5.0000000e+00 6.1344672e-02]
[1.4700000e+02 7.3000000e+01 1.7000000e+01 1.3100436e-02]
[1.4700000e+02 7.2000000e+01 1.0000000e+01 5.1494025e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 3.4214515e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 3.0803775e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.3083703e-02]
[1.470000e+02 7.400000e+01 2.400000e+01 9.354467e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2396586e-02]
[1.4700000e+02 7.4000000e+01 2.3000000e+01 1.9417666e-03]
[1.470000e+02 7.200000e+01 4.000000e+00 6.183122e-02]
[1.47000000e+02 7.30000000e+01 1.80000000e+01 1.13465935e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 5.3064965e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 3.2039355e-02]
[1.4700000e+02 7.3000000e+01 1.7000000e+01 3.3131678e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.1718716e-02]
[1.47000000e+02 7.40000000e+01 2.40000000e+01 1.06966235e-02]
[1.4700000e+02

[1.4700000e+02 7.4000000e+01 2.4000000e+01 1.0348981e-02]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2159795e-02]
[1.4700000e+02 7.4000000e+01 2.3000000e+01 1.8764242e-03]
[1.4700000e+02 7.2000000e+01 4.0000000e+00 6.2082976e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 1.0260337e-02]
[1.4700000e+02 7.2000000e+01 9.0000000e+00 5.4027114e-02]
[1.4700000e+02 7.2000000e+01 1.0000000e+01 3.0528253e-02]
[1.470000e+02 7.300000e+01 1.700000e+01 3.469537e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.0710257e-02]
[1.470000e+02 7.300000e+01 2.400000e+01 1.182749e-02]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.1658464e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 2.0655757e-03]
[1.4700000e+02 7.2000000e+01 4.0000000e+00 6.2287856e-02]
[1.470000e+02 7.300000e+01 1.900000e+01 9.044221e-03]
[1.4700000e+02 7.2000000e+01 8.0000000e+00 5.5084668e-02]
[1.4700000e+02 7.2000000e+01 1.1000000e+01 2.8590873e-02]
[1.470000e+02 7.300000e+01 1.600000e+01 3.665786e-02]
[1.47000e+02 7.20000e+01 4.000

[1.4700000e+02 7.3000000e+01 1.7000000e+01 1.3808236e-02]
[1.470000e+02 7.200000e+01 1.000000e+01 5.085716e-02]
[1.4700000e+02 7.2000000e+01 8.0000000e+00 3.5051968e-02]
[1.4700000e+02 7.3000000e+01 1.9000000e+01 2.9878695e-02]
[1.470000e+02 7.200000e+01 3.000000e+00 5.359848e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 8.8489605e-03]
[1.470000e+02 7.200000e+01 2.000000e+00 6.245195e-02]
[1.4700000e+02 7.4000000e+01 2.3000000e+01 2.0813576e-03]
[1.4700000e+02 7.2000000e+01 4.0000000e+00 6.1661478e-02]
[1.47000000e+02 7.30000000e+01 1.70000000e+01 1.19771315e-02]
[1.470000e+02 7.200000e+01 1.000000e+01 5.250411e-02]
[1.470000e+02 7.200000e+01 9.000000e+00 3.283369e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 3.2294307e-02]
[1.470000e+02 7.200000e+01 3.000000e+00 5.222114e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 1.0208122e-02]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2202536e-02]
[1.470000e+02 7.400000e+01 2.300000e+01 1.873642e-03]
[1.470000e+02 7.200000e+01 4.000000e+0

[1.4700000e+02 7.2000000e+01 7.0000000e+00 4.0193602e-02]
[1.4700000e+02 7.3000000e+01 2.0000000e+01 2.3935081e-02]
[1.470000e+02 7.200000e+01 2.000000e+00 5.658544e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 5.9276344e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2659696e-02]
[1.4700000e+02 7.3000000e+01 2.2000000e+01 3.3853985e-03]
[1.47000e+02 7.20000e+01 5.00000e+00 6.03881e-02]
[1.4700000e+02 7.3000000e+01 1.5000000e+01 1.6301375e-02]
[1.4700000e+02 7.2000000e+01 1.1000000e+01 4.8567757e-02]
[1.4700000e+02 7.2000000e+01 7.0000000e+00 3.7873384e-02]
[1.4700000e+02 7.3000000e+01 2.0000000e+01 2.6695924e-02]
[1.47000e+02 7.20000e+01 2.00000e+00 5.52494e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 7.2282376e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2587075e-02]
[1.4700000e+02 7.4000000e+01 2.2000000e+01 2.6918205e-03]
[1.4700000e+02 7.2000000e+01 5.0000000e+00 6.1019514e-02]
[1.470000e+02 7.300000e+01 1.600000e+01 1.422653e-02]
[1.4700000e+02 7.2000000e+01 1.0000000

[1.4700000e+02 7.4000000e+01 2.4000000e+01 3.7436264e-03]
[1.470000e+02 7.200000e+01 3.000000e+00 6.267046e-02]
[1.4700000e+02 7.3000000e+01 2.1000000e+01 5.4013617e-03]
[1.470000e+02 7.200000e+01 6.000000e+00 5.855618e-02]
[1.4700000e+02 7.3000000e+01 1.3000000e+01 2.1331338e-02]
[1.4700000e+02 7.2000000e+01 1.3000000e+01 4.3815933e-02]
[1.4700000e+02 7.2000000e+01 6.0000000e+00 4.2981617e-02]
[1.470000e+02 7.300000e+01 2.100000e+01 2.061422e-02]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 5.8085777e-02]
[1.4700000e+02 7.4000000e+01 2.4000000e+01 4.5833113e-03]
[1.470000e+02 7.200000e+01 3.000000e+00 6.269189e-02]
[1.4700000e+02 7.3000000e+01 2.1000000e+01 4.3992247e-03]
[1.4700000e+02 7.2000000e+01 6.0000000e+00 5.9525877e-02]
[1.470000e+02 7.300000e+01 1.400000e+01 1.899772e-02]
[1.4700000e+02 7.2000000e+01 1.2000000e+01 4.6026964e-02]
[1.4700000e+02 7.2000000e+01 6.0000000e+00 4.0729884e-02]
[1.4700000e+02 7.3000000e+01 2.1000000e+01 2.3289386e-02]
[1.47000e+02 7.20000e+01 2.00000e+

[1.4700000e+02 7.2000000e+01 1.1000000e+01 4.9818523e-02]
[1.47000e+02 7.20000e+01 8.00000e+00 3.63607e-02]
[1.4700000e+02 7.3000000e+01 1.9000000e+01 2.8423287e-02]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 5.4369945e-02]
[1.47000e+02 7.40000e+01 2.40000e+01 8.09156e-03]
[1.470000e+02 7.200000e+01 2.000000e+00 6.252192e-02]
[1.470000e+02 7.400000e+01 2.300000e+01 2.331634e-03]
[1.4700000e+02 7.2000000e+01 5.0000000e+00 6.1374012e-02]
[1.4700000e+02 7.3000000e+01 1.7000000e+01 1.2998304e-02]
[1.4700000e+02 7.2000000e+01 1.0000000e+01 5.1585943e-02]
[1.470000e+02 7.200000e+01 9.000000e+00 3.409193e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 3.0937733e-02]
[1.4700000e+02 7.2000000e+01 3.0000000e+00 5.3008415e-02]
[1.470000e+02 7.400000e+01 2.400000e+01 9.429331e-03]
[1.4700000e+02 7.2000000e+01 2.0000000e+00 6.2388618e-02]
[1.470000e+02 7.400000e+01 2.300000e+01 1.923886e-03]
[1.4700000e+02 7.2000000e+01 4.0000000e+00 6.1855327e-02]
[1.4700000e+02 7.3000000e+01 1.8000000e+01 1.12571